# Construyendo un chatbot con NLTK desde cero

## Importamos librerías

In [3]:
import io
import random
import string
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'sklearn'

## Importar NLTK 

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/gustavo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Leemos el corpus con el texto para el chatbot

Podemos utilizar una página de Wikipedia para crear el corpus. Podemos copiar el contenido en un fichero de texto.

Debemos procesar el texto para para convertirlo en datos que el algoritmo pueda interpretar. Por ejemplo:

* Convertir el texto a mayúsculas/minúsculas para que el algoritmo no diferencie los token.
* Tokenización: NLTK permite tokenizar por frases (sentence tokenizer) y por palabras (word tokenizer)

El paquete NLTK incluye:

* Listas de stopwords
* Stemming: obtener la raíz de una palabra. Por ejemplo, el algoritmo de Porter es un algoritmo de derivación de palabras y consiste en elimar las términaciones morfológicas comunes de las palabras en inglés, aunque es posible usarlo también en español. Es un proceso de normalización de términos muy usado en corpus basados en texto. Por ejemplo, texto lo transforma a text y canta a cant.
* Lematización: relaciona una palabra flexionada o derivada con su forma canónica o lema. Por ejemplo, cantas, canto lo convierte a cantar. 

In [3]:
f=open('chatbot-es.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

## Tokenization

In [4]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Preprocesando

Analizamos los tokens para normalizarlos

In [5]:
#lemmer = nltk.stem.WordNetLemmatizer()
stemmer = SnowballStemmer('spanish')

def LemTokens(tokens):
    #return [lemmer.lemmatize(token) for token in tokens]
    return [stemmer.stem(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

Definimos una función para saludar.

In [6]:
GREETING_INPUTS = ("hola", "que tal?")
GREETING_RESPONSES = ["Buenos días", "hola"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### Generamos la respuesta

### Bag of Words

Después del preproceso inicial, transformamos el texto en un vector (array) de números. El bag-of-words es la representación del texto que describe las ocurrencias de las palabras en un documento incluyendo el vocabulario de palabras y el número de ocurrencias. No se tiene en cuenta ni dónde ocurren ni el orden, solo tiene en cuenta si las palabras aparecen en el texto.

En el caso de bag-of-words los documentos se consideran similares si contienen texto similar. Por ejemplo, {Gustavo, juega, al, fútbol} y vectorizamos el texto "Gustavo juega", tendríamos el siguiente vector: (1,1,0,0).


**TF-IDF**

Term Frequency: la frecuenca de las palabras en un documento.

TF = (Número de veces que el término t aparece en un documento)/(Número de términos del documento)

Inverse Document Frequency: mide la relevancia de una palabra para un documento en una colección.

IDF = 1+log(N/n), donde, N el número de documentos y n es el número de documentos donde el térmimo t aparece.


**Similitud del coseno**
Es una medida de la similitud existente entre dos vectores. Mide el ángulo entre dos vectores.



Para generar una respuesta los conceptos de similitud se tienen en cuenta. Definimos una función de respuesta que busca una o más palabras y devuelve una posible respuesta. Si no encuentra nada devuelve: ”Lo siento! No entiendo tu pregunta”

In [7]:
def response(user_response):
    response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words("spanish"))
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        response=robo_response+"Lo siento! No entiendo tu pregunta"
        return response
    else:
        response = response+sent_tokens[idx]
        return response

Finalmente, añadimos una función para solicitar el texto por pantalla.

In [ ]:
flag=True
print("ATI-UA: Mi nombre is ATI-UA y puedo contestar preguntas relacionadas con chatbots. Si deseas salir, inserta Adios!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='adios'):
        if(user_response=='Gracias' or user_response=='Muchas gracias' ):
            flag=False
            print("ATI-UA: De nada.")
        else:
            if(greeting(user_response)!=None):
                print("ATI-UA: "+greeting(user_response))
            else:
                print("ATI-UA: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ATI-UA: Hasta luego!")

ATI-UA: Mi nombre is ATI-UA y puedo contestar preguntas relacionadas con chatbots. Si deseas salir, inserta Adios!
chatbot
ATI-UA: estos servicios por chatbot ya existían por varios años en la app wechat.


#  Referencias

* https://github.com/parulnith/Building-a-Simple-Chatbot-in-Python-using-NLTK/blob/master/Chatbot.ipynb
* https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e
* [Intro NLP](http://josearcosaneas.github.io/python/r/procesamiento/lenguaje/2017/01/02/procesamiento-lenguaje-natural-0.html#:~:text=El%20stemming%20consiste%20en%20extreaer,ayuda%20de%20la%20librer%C3%ADa%20NLTK.)